'll help write out the mathematical notation for convolutions in CNNs and their backpropagation. Let me use similar notation style to what you provided for clear comparison.
For a convolutional layer, the forward pass can be written as:
$$z_{i,k} = \sum_{c=1}^{C_{i-1}} (A_{i-1,c} * W_{i,k}) + b_{i,k}$$
$$A_{i,k} = f_i(z_{i,k})$$
Where:

$k$ indexes the output channel/feature map
$c$ indexes the input channel
$*$ denotes the convolution operation
$C_{i-1}$ is the number of channels in layer $i-1$
$W_{i,k}$ is the $k$-th filter/kernel in layer $i$
$b_{i,k}$ is the bias for the $k$-th feature map

For backpropagation through a convolutional layer:
$$\partial z_{i,k} = \begin{cases}
A_L - Y & i = L \
\sum_{k'} (\partial z_{i+1,k'} * \text{rot180}(W_{i+1,k'})) \odot f'i(z{i,k}) & i < L
\end{cases}$$
$$\partial W_{i,k} = \sum_{n=1}^N \frac{A_{i-1} * \partial z_{i,k}}{N}$$
$$\partial b_{i,k} = \sum_{h,w} \frac{\partial z_{i,k}(h,w)}{H \times W}$$
Where:

$\text{rot180}(W)$ is the 180-degree rotation of the filter $W$
$\odot$ represents element-wise multiplication
$N$ is the batch size
$H,W$ are the height and width of the feature map
The convolution in $\partial W_{i,k}$ is actually a cross-correlation
The summation over $k'$ in $\partial z_{i,k}$ is over all filters in layer $i+1$ that use the $k$-th feature map as input

Note that these formulas assume "valid" convolutions. For "same" padding, additional terms would be needed to account for the padding operations.

In [ ]:
from layers import Layer, DenseLayer
from nn import NN
from tensors import Vector, Matrix


class ConvolvingLayer(Layer):
    name = "Convolving Layer"
    
    def __init__(self, input_shape: tuple[int, int], kernels: list[Matrix], activation_func: Function = Identity, /,
        padding_value: float = 0, stide: tuple[int, int] = (1, 1))
        self.kernels = kernels
        self.shape = len(kernels), *kernels[0].shape
        self.activ = activation_func
    
        self.padding_value = padding_value
        self.stride = stride

        self.z = None
        self.A = None
    
        # Used for sum, don't have to init a new matrix for every forward prop this way.
        feature_maps_shape = input_shape[0] - kernels[0].shape[0] + 1, input_shape[1] - kernels[0].shape[1] + 1
        self.zero_feature = Matrix.filled(0, *feature_maps_shape)

    def predict(self, prev_A: Matrix) -> Matrix:
        self.z = [prev_A.convolve(k) for k in self.kernels]
        self.A = self.activ.compute(reduce(operator.add, feature_maps, self.zero_feature))

        return self.A
    
    def backprop(self, next_layer: Matrix) -> Matrix:
        pass

    def gd(self, prev_A: Matrix, alpha: float, n: int) -> Matrix:
        pass
        
    def __repr__(self):
        kernel_str = "\n\n".join([str(k) for k in self.kernels])
        
        return f"({self.name}, {l.activ.name})\n" \
               f"   shape:\n{self.shape}\n" \
               f"   kernels:\n{kernel_str}\n\n" 
        

class PoolingLayer(Layer):
    name = "Pooling Layer"
    hows = ['max', 'avg']
    
    def __init__(self, input_shape: tuple[int, int], stride: tuple[int, int], how = 'avg'):
        self.stride = stride
        self.how = how
        
    def predict(self, prev_A: Matrix) -> Matrix:
        pass
    
    def backprop(self, next_layer: Matrix) -> Matrix:
        pass

    def gd(self, prev_A: Matrix, alpha: float, n: int) -> Matrix:
        pass

    def __repr__(self):
        return f"({self.name}, {self.stride[0]}x{self.stride[1]} {self.how})\n\n"
               

class FlatteningLayer(Layer):
    name = "Flattening Layer"
    
    def __init__(self, n_rows_in: int):
        self.n_rows_in = n_rows_in
    
    def predict(self, prev_A: Matrix) -> Vector:
        return Vector([x for x in v for v in prev_A])

    def backprop(self, next_layer: Vector) -> Matrix:
        interval = len(next_layer) // self.n_rows_in
        
        def batch(l) -> list:
            return [l[i:i+interval] for i in range(0, len(l), interval)]

        return Matrix(batch(next_layer))

    def gd(self, prev_A: Matrix, alpha: float, n: int) -> Vector:
        return self.predict(prev_A)
        
    def __repr__(self):
        return f"({l.name})\n\n"

# LAYER SHAPE CHECKS
        
# SOMETHING EFFECTIVE RECEPTIVE FIELD CALC
    # Backprop to input + heatmap
